In [1]:
import geemap.eefolium as geemap
import ee

# trick: AttributeError: module 'collections' has no attribute 'Callable'
import collections
collections.Callable = collections.abc.Callable

ee.Initialize()

In [2]:
# initialise some variables
center = [51.2303,  -0.6344]
zoom = 16
large_chalk_shp = 'projects/ee-surrey-svocd/assets/Blackwell_01_LargeChalk_shp'
oak_shp = 'projects/ee-surrey-svocd/assets/Blackwell_06_Oak_shp'
dairy_farm_shp = 'projects/ee-surrey-svocd/assets/Blackwell_08_DairyFarm_shp'
uos_property_shp = 'projects/ee-surrey-svocd/assets/UoS_Property_shp'

In [3]:
# get sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')

In [4]:
#gee assets to get the AOI
large_chalk_area = ee.FeatureCollection(large_chalk_shp)
oak_area = ee.FeatureCollection(oak_shp)
dairy_farm_area = ee.FeatureCollection(dairy_farm_shp)
uos_property_area = ee.FeatureCollection(uos_property_shp)

In [5]:
# get AOI images
large_chalk_area_image = s2.filterBounds(large_chalk_area).filterDate('2021-01-01', '2021-04-30').sort('CLOUDY_PIXEL_PERCENTAGE', False).mosaic().clip(large_chalk_area)
oak_area_image = s2.filterBounds(oak_area).filterDate('2021-01-01', '2021-04-30').sort('CLOUDY_PIXEL_PERCENTAGE', False).mosaic().clip(oak_area)
dairy_farm_area_image = s2.filterBounds(dairy_farm_area).filterDate('2021-01-01', '2021-04-30').sort('CLOUDY_PIXEL_PERCENTAGE', False).mosaic().clip(dairy_farm_area)
uos_property_area_image = s2.filterBounds(uos_property_area).filterDate('2021-01-01', '2021-04-30').sort('CLOUDY_PIXEL_PERCENTAGE', False).mosaic().clip(uos_property_area)

In [6]:
# set image true color bands
true_color = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

In [7]:
# view UoS property on map
map = geemap.Map(center=[51.2355,  -0.6270], zoom=15, max_zoom=16, min_zoom=14)
map.setOptions('SATELLITE')
map.addLayer(uos_property_area_image, true_color, 'Sentinel-2 True Color')
map.addLayerControl()
map

In [8]:
# calculate the satellite indices
ndwi = uos_property_area_image.normalizedDifference(['B3', 'B8']).rename('NDWI')
# ndbi = uos_property_area_image.normalizedDifference(['B3', 'B2']).rename('NDBI')
# ndvi = uos_property_area_image.normalizedDifference(['B5', 'B4']).rename('NDVI')

ndwi_color = ['red', 'yellow', 'green', 'blue']


In [11]:
# display ndwi on map
map = geemap.Map(center=[51.2355,  -0.6270], zoom=14)
map.setOptions('SATELLITE')
map.addLayer(ndwi, {'min': -1, 'max': 0.5, 'palette': ndwi_color}, 'NDWI')
# map.addLayerControl()
map